
## Simulation of the LHC 6L2 module with Wakis

| Result                                                                                                                                                                  | Mesh and parameters | Model                                                                                                                                                                                                                                                       |
| ----------------------------------------------------------------------------------------------------------------------------------------------------------------------- | ------------------- | ----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| ![](https://codimd.web.cern.ch/uploads/upload_4494dc3d0aac49d88cb5b49e4926f53d.png) ![](https://codimd.web.cern.ch/uploads/upload_f9b4e53e2ddda4403d84acd70c58f670.png) |        ![](https://codimd.web.cern.ch/uploads/upload_7cc0b2faab05035ce9a14e3b3b0702ff.PNG)             | ![](https://codimd.web.cern.ch/uploads/upload_8571bf9c24f603c1f787f84e078d94f1.png) ![](https://codimd.web.cern.ch/uploads/upload_35a9cc226c5d945c2137c740db643c49.png) ![](https://codimd.web.cern.ch/uploads/upload_d4e0937c57ff04ac1a796a600798bd05.png) |


### Generate the STL files from STEP
> Needs to use cadquery venv

In [ ]:
# test cadquery import
import cadquery as cq
cq.__version__

In [ ]:
from wakis import geometry
stp_file = '012_VSMIO.stp'

In [ ]:
# Extract dictionaries with names of solids and materials
stl_solids = geometry.extract_solids_from_stp(stp_file)
print(f'Found {len(stl_solids)} solids in the STP file.')
print(stl_solids)
stl_colors = geometry.extract_colors_from_stp(stp_file)
print(f'Found {len(stl_colors)} colors in the STP file.')
print(stl_colors)
stl_materials = geometry.extract_materials_from_stp(stp_file)
print(f'Found {len(stl_materials)} materials in the STP file.')
print(stl_materials)
unit = geometry.get_stp_unit_scale(stp_file)
print(f'STP unit scale: {unit} m')

In [ ]:
stl_solids = geometry.generate_stl_solids_from_stp(stp_file)

### Combine STL files by material

In [ ]:
import pyvista as pv
import numpy as np
from wakis import geometry
stp_file = '012_VSMIO.stp'

# Extract dictionaries with names of solids and materials
stl_solids = geometry.extract_solids_from_stp(stp_file, path='stl')
print(f'Found {len(stl_solids)} solids in the STP file.')
print(stl_solids)
stl_colors = geometry.extract_colors_from_stp(stp_file)
print(f'Found {len(stl_colors)} colors in the STP file.')
print(stl_colors)
stl_materials = geometry.extract_materials_from_stp(stp_file)
print(f'Found {len(stl_materials)} materials in the STP file.')
print(stl_materials)
unit = geometry.get_stp_unit_scale(stp_file)
print(f'STP unit scale: {unit} m')

In [ ]:
materials = np.unique(list(stl_materials.values()))
print(f'Unique materials: {materials}')

colors = np.unique(list(stl_colors.values()), axis=0)
print(f'Unique colors: {colors}')

new_stl_colors = {}

In [ ]:
# Combine STL files by material
material = '316l'
aux = None
for key, mat in stl_materials.items():
    if mat.lower() == material.lower():
        color = stl_colors[key]
        print(f"Adding solid: {key.split('_')[0]} with material: {mat}")
        if aux is None:
            aux = pv.read(stl_solids[key]).triangulate()
        else:
            surf = pv.read(stl_solids[key]).triangulate()
            aux = aux + surf
surf = aux
del aux
surf.save(f'012_LHC_WVM_6L2-{material}.stl')
surf.plot(color=color, silhouette=True)
new_stl_colors[material] = color

In [ ]:
material = 'cube'
aux = None
for key, mat in stl_materials.items():
    if mat.lower() == material.lower():
        print(f"Adding solid: {key.split('_')[0]} with material: {mat}")
        if aux is None:
            color = stl_colors[key]
            aux = pv.read(stl_solids[key]).triangulate()
        else:
            surf = pv.read(stl_solids[key]).triangulate()
            aux = aux + surf
surf = aux
del aux
surf.save(f'012_LHC_WVM_6L2-{material}.stl')
surf.plot(color=color, silhouette=True)
new_stl_colors[material] = color

In [ ]:
material = 'cucamera'
aux = None
for key, mat in stl_materials.items():
    if mat.lower() == material.lower():
        print(f"Adding solid: {key.split('_')[0]} with material: {mat}")
        if aux is None:
            color = stl_colors[key]
            aux = pv.read(stl_solids[key]).triangulate()
        else:
            surf = pv.read(stl_solids[key]).triangulate()
            aux = aux + surf
surf = aux
del aux
surf.save(f'012_LHC_WVM_6L2-{material}.stl')
surf.plot(color=color, silhouette=True)
new_stl_colors[material] = color

In [ ]:
material = 'martensite'
aux = None
for key, mat in stl_materials.items():
    if mat.lower() == material.lower():
        print(f"Adding solid: {key.split('_')[0]} with material: {mat}")
        if aux is None:
            color = stl_colors[key]
            aux = pv.read(stl_solids[key]).triangulate()
        else:
            surf = pv.read(stl_solids[key]).triangulate()
            aux = aux + surf
surf = aux
del aux
surf.save(f'012_LHC_WVM_6L2-{material}.stl')
surf.plot(color=color, silhouette=True)
new_stl_colors[material] = color

In [ ]:
print('New STL colors dictionary:')
print(new_stl_colors)

### Generate simulation mesh

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pyvista as pv

from wakis import SolverFIT3D
from wakis import GridFIT3D 
from wakis import WakeSolver
from wakis import geometry

%matplotlib ipympl

In [ ]:
# ---------- Geometry setup ---------
# Extract information from STEP file
stp_file = '012_VSMIO.stp'

stp_materials= geometry.extract_materials_from_stp(stp_file)
stp_colors = geometry.extract_colors_from_stp(stp_file)

materials = np.unique(list(stp_materials.values()))[:-1]
colors = np.unique(list(stp_colors.values()), axis=0)

print(f'Materials in STEP model: {materials}')
print(f'Colors in STEP model: {colors}')

materials = {'316l': [1e4, 1., 1e4], 
            'cube': [1e4, 1., 1e4], 
            'cucamera': [1e4, 1., 1e4],
            'martensite': [1e4, 1., 1e4]}

# Input dictionaries
basename = '012_LHC_WVM_6L2'
stl_solids, stl_materials = {}, {}
for material in materials:
    print(f'Processing material: {material}')
    stl_solids[material] = f'{basename}-{material}.stl'
    stl_materials[material] = materials[material]

stl_colors = {'316l': [1.,1.,1.], 
            'cube':  [0.82745099, 0.698039, 0.49019599],  
            'cucamera': [0.82352901, 0.82352901, 1. ],  
            'martensite': [0.,1.,1.], }

In [ ]:
# ---------- Domain mesh setup ---------
# Number of mesh cells
Nx = 100//2
Ny = 100//2
Nz = 200//2

# Domain bounds
surf = None
for stl in stl_solids.values():
    aux = pv.read(stl)
    if surf is None:
        surf = aux
    else:
        surf = surf + aux

xmin, xmax, ymin, ymax, zmin, zmax = surf.bounds
Lx, Ly, Lz = (xmax-xmin), (ymax-ymin), (zmax-zmin)

# set grid and geometry
grid = GridFIT3D(xmin, xmax, ymin, ymax, zmin, zmax, 
                Nx, Ny, Nz, 
                stl_solids=stl_solids, 
                stl_materials=stl_materials,
                stl_colors=stl_colors,
                )

In [ ]:
# ------------ Beam source ----------------
# Beam parameters
sigmaz = 50e-3      #[m] -> 2 GHz
q = 1e-9            #[C]
beta = 1.0          # beam beta 
xs = 0.             # x source position [m]
ys = 0.             # y source position [m]
xt = 0.             # x test position [m]
yt = 0.             # y test position [m]

# ----------- Solver  setup  ----------
# Wakefield post-processor
wakelength = 10. #[m]
skip_cells = 20  # no. cells
results_folder = 'results/'
wake = WakeSolver(q=q, sigmaz=sigmaz, beta=beta,
                xsource=xs, ysource=ys, xtest=xt, ytest=yt,
                skip_cells=skip_cells,
                results_folder=results_folder,
                Ez_file=results_folder+'Ez.h5',)

In [ ]:
# boundary conditions
bc_low=['pec', 'pec', 'pml']
bc_high=['pec', 'pec', 'pml']
n_pml = 15

solver = SolverFIT3D(grid, wake,
                     bc_low=bc_low, bc_high=bc_high, 
                     n_pml=n_pml,
                     use_stl=True, 
                     dtype=np.float32,
                     bg='pec')

solver.logger.save_logs()

In [ ]:
pl = solver.inspect(specular=0.5, 
                    add_silhouette=True, 
                    off_screen=False)

In [ ]:
pl.export_html('results/solver.html')